In [2]:
%matplotlib qt
#inline
import os
import os.path as op
import mne
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from mne.time_frequency import (tfr_multitaper, tfr_stockwell, tfr_morlet,
                                tfr_array_morlet)
from mne.stats import f_threshold_mway_rm, f_mway_rm, fdr_correction,bonferroni_correction
from mne.channels import find_ch_adjacency
from mne.stats import spatio_temporal_cluster_test, permutation_t_test
import scipy.stats as stats
import statsmodels.api as sm
import seaborn as sns

C:\Users\amin\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
#leading preprocessed eeg
list_subj = ['sub-001', 'sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-008', 'sub-009',
             'sub-010', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017']

# define evoked for each condition
# 437 points for 1700 ms
evoked_74 = np.zeros((14,62,437))
evoked_75 = np.zeros((14,62,437))
evoked_76 = np.zeros((14,62,437))
idx =  0
for i in list_subj:
    examples_dir = "PDM_data\\"+i+"\\sourcedata-eeg_outside-MRT\\eeg";
    epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");
    epochs.info['bads'] = []
    #resample epochs.resample(128)
    evoked_74[idx] = epochs['74'].average().data
    evoked_75[idx] = epochs['75'].average().data
    evoked_76[idx] = epochs['76'].average().data
    
    idx = idx + 1

<ipython-input-4-207133ead15d>:13: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-001\sourcedata-eeg_outside-MRT\eeg\sub-001_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-001\sourcedata-eeg_outside-MRT\eeg\sub-001_prerprocessed.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
275 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-003\sourcedata-eeg_outside-MRT\eeg\sub-003_prerprocessed.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active


<ipython-input-4-207133ead15d>:13: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-003\sourcedata-eeg_outside-MRT\eeg\sub-003_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
280 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-004\sourcedata-eeg_outside-MRT\eeg\sub-004_prerprocessed.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available


<ipython-input-4-207133ead15d>:13: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-004\sourcedata-eeg_outside-MRT\eeg\sub-004_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


...

Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-016\sourcedata-eeg_outside-MRT\eeg\sub-016_prerprocessed.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
245 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated


<ipython-input-4-207133ead15d>:13: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-016\sourcedata-eeg_outside-MRT\eeg\sub-016_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-017\sourcedata-eeg_outside-MRT\eeg\sub-017_prerprocessed.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
251 matching events found
Applying baseline correction (mode: mean)


<ipython-input-4-207133ead15d>:13: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-017\sourcedata-eeg_outside-MRT\eeg\sub-017_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


Created an SSP operator (subspace dimension = 1)
1 projection items activated


# Amplitude at Central electrods

In [5]:
#selected channels from centrals sites
ch_name_C1 ='C1'
ch_name_C2 ='C2'
channel_indices_C1 = mne.pick_channels(epochs.info['ch_names'], [ch_name_C1])[0]
channel_indices_C2 = mne.pick_channels(epochs.info['ch_names'], [ch_name_C2])[0]

ch_name_C3 ='C3'
ch_name_C4 ='C4'
channel_indices_C3 = mne.pick_channels(epochs.info['ch_names'], [ch_name_C3])[0]
channel_indices_C4 = mne.pick_channels(epochs.info['ch_names'], [ch_name_C4])[0]


# setting times
times = epochs.times

In [6]:
#defining ipislateral and neutral activity
ipsilateral_amplitude   = np.array((evoked_74[:,channel_indices_C1,:],evoked_75[:,channel_indices_C2,:],
                                    evoked_74[:,channel_indices_C3,:],evoked_75[:,channel_indices_C4,:])).mean(axis = 0)

neutral_amplitude       = np.array((evoked_76[:,channel_indices_C1,:],evoked_76[:,channel_indices_C2,:],
                                    evoked_76[:,channel_indices_C3,:],evoked_76[:,channel_indices_C4,:])).mean(axis = 0)

In [9]:
#plot contalateral and neutral

plt.figure()

plt.plot(times*1000,ipsilateral_amplitude.mean(axis=0),
         label='Ipsi')
error = ipsilateral_amplitude.std(axis=0)/np.sqrt(14)
plt.fill_between(times*1000, ipsilateral_amplitude.mean(axis=0) -error, ipsilateral_amplitude.mean(axis=0) +error,
    alpha=0.3)


plt.plot(times*1000,neutral_amplitude.mean(axis=0),
          label='Neut')
error = neutral_amplitude.std(axis=0)/np.sqrt(14)

plt.fill_between(times*1000, neutral_amplitude.mean(axis=0) -error, neutral_amplitude.mean(axis=0) +error,
    alpha=0.3)


plt.plot(times*1000,ipsilateral_amplitude.mean(axis=0) - neutral_amplitude.mean(axis=0),
         'k--', label='Ipsi - Neut')

plt.plot(times*1000,np.zeros(len(times)),'y') 

plt.legend()
plt.show()
sns.despine(offset=10, trim=True)  # <-- only show the right spine

plt.axvline(x=0, color='black')
plt.axvline(x=1000, color='black')

In [10]:
#setting time window
win_low = int((100 + 1150)/3.9)
win_up  = int((100 + 1500)/3.9)

In [11]:
win_low,win_up

(320, 410)

In [12]:
#### ttest and fdr correction
T, pval = stats.ttest_rel(ipsilateral_amplitude[:,win_low:win_up], neutral_amplitude[:,win_low:win_up])
alpha = 0.05

reject_fdr, pval_fdr = fdr_correction(pval, alpha=alpha, method='indep')

In [13]:
#time point in Ms which are significant based on 
3.9*np.arange(win_low,win_up)[pval_fdr<0.05]-100

array([1155.8, 1159.7, 1163.6, 1167.5, 1171.4, 1210.4, 1214.3, 1218.2,
       1222.1, 1226. , 1229.9, 1233.8, 1237.7, 1241.6, 1245.5, 1249.4,
       1253.3, 1257.2, 1261.1, 1265. , 1268.9])

In [15]:
time_min1 = int((100 + 1210)/3.9)
time_max1 = int((100 + 1270)/3.9)

In [16]:
#fill areas which are significant
plt.axvspan(times[time_min1]*1000, times[time_max1]*1000, color=sns.xkcd_rgb['grey'], alpha=0.4, ymax= 0.66)

In [15]:
#average amplitude
ipsilateral_amplitude_mean = ipsilateral_amplitude[:,time_min1:time_max1].mean(axis = 1)
neutral_amplitude_mean   = neutral_amplitude[:,time_min1:time_max1].mean(axis = 1)

In [16]:
#t-test average amplitude
stats.ttest_rel(ipsilateral_amplitude_mean,neutral_amplitude_mean)

Ttest_relResult(statistic=-4.712199835902783, pvalue=0.0004061744795015991)

In [17]:
diff_amplitude_mean = ipsilateral_amplitude_mean - neutral_amplitude_mean

# Alpha Power at Central electrod

In [17]:
#define frequencies of interest
freqs = np.arange(4, 30, 1)  
#n_cycles =freqs/2
n_cycles =np.arange(3, 16, 0.5)
n_freqs = len(freqs)

In [18]:
#### two selected channels
ch_name_PO3 ='PO3'
ch_name_PO4 ='PO4'
channel_indices_PO3 = mne.pick_channels(epochs.info['ch_names'], [ch_name_PO3])[0]
channel_indices_PO4 = mne.pick_channels(epochs.info['ch_names'], [ch_name_PO4])[0]

ch_name_PO7 ='PO7'
ch_name_PO8 ='PO8'
channel_indices_PO7 = mne.pick_channels(epochs.info['ch_names'], [ch_name_PO7])[0]
channel_indices_PO8 = mne.pick_channels(epochs.info['ch_names'], [ch_name_PO8])[0]


# setting times
times = epochs.times

In [19]:
#single trial wavelet
list_subj = ['sub-001', 'sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-008', 'sub-009',
             'sub-010', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017']

# define evoked for each condition
# 437 points for 1700 ms
vawelet_74_PO3 = np.zeros((14,n_freqs,437))
vawelet_74_PO4 = np.zeros((14,n_freqs,437))
vawelet_75_PO3 = np.zeros((14,n_freqs,437))
vawelet_75_PO4 = np.zeros((14,n_freqs,437))
vawelet_76_PO3 = np.zeros((14,n_freqs,437))
vawelet_76_PO4 = np.zeros((14,n_freqs,437))



vawelet_74_PO7 = np.zeros((14,n_freqs,437))
vawelet_74_PO8 = np.zeros((14,n_freqs,437))
vawelet_75_PO7 = np.zeros((14,n_freqs,437))
vawelet_75_PO8 = np.zeros((14,n_freqs,437))
vawelet_76_PO7 = np.zeros((14,n_freqs,437))
vawelet_76_PO8 = np.zeros((14,n_freqs,437))

idx =  0
for i in list_subj:
    examples_dir = "PDM_data\\"+i+"\\sourcedata-eeg_outside-MRT\\eeg";
    epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");
    #this code very important
    epochs.info['bads'] = []

    #define frequencies of interest
    
    #74
    power = tfr_morlet(epochs['74'], freqs, n_cycles=n_cycles, average = False,
                              return_itc=False)  
    power = power.average()
    vawelet_74_PO3[idx] = power.data[channel_indices_PO3, :, :]
    vawelet_74_PO4[idx] = power.data[channel_indices_PO4, :, :]
    
    vawelet_74_PO7[idx] = power.data[channel_indices_PO7, :, :]
    vawelet_74_PO8[idx] = power.data[channel_indices_PO8, :, :]
    
    #75
    power = tfr_morlet(epochs['75'], freqs, n_cycles=n_cycles, average = False,
                              return_itc=False)  
    power = power.average()
    vawelet_75_PO3[idx] = power.data[channel_indices_PO3, :, :]
    vawelet_75_PO4[idx] = power.data[channel_indices_PO4, :, :]
    
    
    vawelet_75_PO7[idx] = power.data[channel_indices_PO7, :, :]
    vawelet_75_PO8[idx] = power.data[channel_indices_PO8, :, :]
       
    #76
    power = tfr_morlet(epochs['76'], freqs, n_cycles=n_cycles, average = False,
                              return_itc=False) 
    power = power.average() 
    vawelet_76_PO3[idx] = power.data[channel_indices_PO3, :, :]
    vawelet_76_PO4[idx] = power.data[channel_indices_PO4, :, :]
    
    
    vawelet_76_PO7[idx] = power.data[channel_indices_PO7, :, :]
    vawelet_76_PO8[idx] = power.data[channel_indices_PO8, :, :]
    
    idx = idx + 1

Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-001\sourcedata-eeg_outside-MRT\eeg\sub-001_prerprocessed.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available


<ipython-input-19-26a0662b98ab>:25: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-001\sourcedata-eeg_outside-MRT\eeg\sub-001_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


Not setting metadata
Not setting metadata
275 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Not setting metadata
Not setting metadata
Not setting metadata
Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-003\sourcedata-eeg_outside-MRT\eeg\sub-003_prerprocessed.fif ...


<ipython-input-19-26a0662b98ab>:25: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-003\sourcedata-eeg_outside-MRT\eeg\sub-003_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
280 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Not setting metadata
Not setting metadata
Not setting metadata
Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-004\sourcedata-eeg_outside-MRT\eeg\sub-004_prerprocessed.fif ...


...

    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
257 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Not setting metadata
Not setting metadata
Not setting metadata
Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-016\sourcedata-eeg_outside-MRT\eeg\sub-016_prerprocessed.fif ...


<ipython-input-19-26a0662b98ab>:25: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-016\sourcedata-eeg_outside-MRT\eeg\sub-016_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
245 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Not setting metadata
Not setting metadata
Not setting metadata
Reading D:\PhD\PDM_data\eeg_outside-MRT\sub-017\sourcedata-eeg_outside-MRT\eeg\sub-017_prerprocessed.fif ...


<ipython-input-19-26a0662b98ab>:25: RuntimeWarning: This filename (D:\PhD\PDM_data\eeg_outside-MRT\sub-017\sourcedata-eeg_outside-MRT\eeg\sub-017_prerprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(examples_dir + "\\"+ i + "_prerprocessed.fif");


    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -101.56 ...    1601.56 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
251 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Not setting metadata
Not setting metadata
Not setting metadata


In [20]:
#make a mean of alpha power from 8 Hz to 13 Hz
vawelet_74_PO3_alpha = vawelet_74_PO3[:,4:9,:].mean(axis = 1)
vawelet_74_PO4_alpha = vawelet_74_PO4[:,4:9,:].mean(axis = 1)
vawelet_75_PO3_alpha = vawelet_75_PO3[:,4:9,:].mean(axis = 1)
vawelet_75_PO4_alpha = vawelet_75_PO4[:,4:9,:].mean(axis = 1)
vawelet_76_PO3_alpha = vawelet_76_PO3[:,4:9,:].mean(axis = 1)
vawelet_76_PO4_alpha = vawelet_76_PO4[:,4:9,:].mean(axis = 1)

#make a mean of alpha power from 8 Hz to 13 Hz
vawelet_74_PO7_alpha = vawelet_74_PO7[:,4:9,:].mean(axis = 1)
vawelet_74_PO8_alpha = vawelet_74_PO8[:,4:9,:].mean(axis = 1)
vawelet_75_PO7_alpha = vawelet_75_PO7[:,4:9,:].mean(axis = 1)
vawelet_75_PO8_alpha = vawelet_75_PO8[:,4:9,:].mean(axis = 1)
vawelet_76_PO7_alpha = vawelet_76_PO7[:,4:9,:].mean(axis = 1)
vawelet_76_PO8_alpha = vawelet_76_PO8[:,4:9,:].mean(axis = 1)

In [21]:
#defining ipsilateral and neutral power
ipsilateral_power   = np.array((vawelet_74_PO3_alpha ,vawelet_75_PO4_alpha,
                                vawelet_74_PO7_alpha ,vawelet_75_PO8_alpha)).mean(axis = 0)


neutral_power       = np.array((vawelet_76_PO3_alpha ,vawelet_76_PO4_alpha,
                                vawelet_76_PO7_alpha ,vawelet_76_PO8_alpha)).mean(axis = 0)

In [54]:
#plot ipsilateral and neutral
plt.figure()

plt.plot(times*1000,ipsilateral_power.mean(axis=0),
          label='IpsiIpsi')
error = ipsilateral_power.std(axis=0)/np.sqrt(14)
plt.fill_between(times*1000, ipsilateral_power.mean(axis=0) -error, ipsilateral_power.mean(axis=0) +error,
    alpha=0.2)



plt.plot(times*1000,neutral_power.mean(axis=0),
          label='Neut')
error = neutral_power.std(axis=0)/np.sqrt(14)
plt.fill_between(times*1000, neutral_power.mean(axis=0) -error, neutral_power.mean(axis=0) +error,
    alpha=0.2)


plt.plot(times*1000,ipsilateral_power.mean(axis=0) - neutral_power.mean(axis=0),
         'k-.', label='Ani')

plt.plot(times*1000,np.zeros(len(times)),'y') 
sns.despine(offset=10, trim=True)  # <-- only show the right spine

plt.axvline(x=0, color='black')
plt.axvline(x=1000, color='black')
plt.show()

In [24]:
#Calculating FAL
time_min1 = int((100 + 900)/3.9)
time_max1 = int((100 + 1600)/3.9)
half_area = (ipsilateral_power - neutral_power).mean(axis = 0)[time_min1:time_max1].sum()/2

In [25]:
#50% of FAL
time_min1 = int((100 + 900)/3.9)
time_max1 = int((100 + 1205)/3.9)
new_area = (ipsilateral_power - neutral_power).mean(axis = 0)[time_min1:time_max1].sum()

In [26]:
1e9*half_area - 1e9*new_area

0.011544955743264751

In [27]:
1205-50, 1205+50

(1155, 1255)

In [4]:
#setting time window
time_min1 = int((100 + 1205-50)/3.9)
time_max1  = int((100 + 1205+50)/3.9)

In [5]:
time_min1,time_max1

(321, 347)

In [51]:
plt.axvspan(times[time_min1]*1000, times[time_max1]*1000, color='gray', alpha=0.4, ymin= 0.12)

In [30]:
#averaging power from 1100 to 1200 ms
ipsilateral_power_mean = ipsilateral_power[:,time_min1:time_max1].mean(axis = 1)
neutral_power_mean       = neutral_power[:,time_min1:time_max1].mean(axis = 1)

In [17]:
#t-test of averaging
stats.ttest_rel(ipsilateral_power_mean,neutral_power_mean)

Ttest_relResult(statistic=2.2726215472852087, pvalue=0.040670724568188876)

In [18]:
#difference contralaterla and ipsilateral 
diff_power_mean = (ipsilateral_power_mean - neutral_power_mean)

In [19]:
diff_power_mean

array([-2.60308331e-11,  3.52896500e-12,  1.24066196e-10,  6.26648163e-11,
        3.57662275e-11,  7.15530239e-11,  9.72277218e-11,  8.74521739e-11,
        1.07909579e-11,  2.91667456e-11, -3.91592323e-11, -1.99011880e-11,
        2.93344731e-10, -4.29100206e-12])

In [53]:
#plot time frequency ipsilateral and neutral power
ipsilateral_power_imshow   = np.array((vawelet_74_PO3 ,vawelet_75_PO4,
                                vawelet_74_PO7 ,vawelet_75_PO8)).mean(axis = 0)


neutral_power_imshow        = np.array((vawelet_76_PO3 ,vawelet_76_PO4,
                                vawelet_76_PO7 ,vawelet_76_PO8)).mean(axis = 0)

plt.figure()
diff = (ipsilateral_power_imshow-neutral_power_imshow).mean(axis = 0)
vmax = np.max(diff)
vmin = -vmax
plt.imshow(diff, cmap=plt.cm.RdBu_r,
           extent=[times[0]*1000, times[-1]*1000, freqs[0], freqs[-1]],
           aspect='auto', origin='lower', vmin=vmin, vmax=vmax)
plt.colorbar()

plt.axvline(x=0, color='black')
plt.axvline(x=1000, color='black')
sns.despine(offset=10)  # <-- only show the right spine


# DDM model

In [18]:
#loading modelt csv(non-decision time)
model = pd.read_csv('..\\modelt\\modelt.csv')

In [19]:
#difference of prioritized minus non-prioritized non-decision time
par_yes = model.filter(regex=("t_subj\(Yes\).*")).to_numpy()[0]
par_no  = model.filter(regex=("t_subj\(No\).*")).to_numpy()[0]
par_t   =  par_yes-par_no

In [22]:
reg_independent_diff   = pd.DataFrame({'erp'   : diff_amplitude_mean,
                                       'alpha' : diff_power_mean})

In [21]:
diff_power_mean = np.array([-2.603083e-11,3.528965e-12,1.240662e-10,6.266482e-11,3.576623e-11,
                           7.155302e-11,9.722772e-11,8.745217e-11,1.079096e-11,2.916675e-11,
                           -3.915923e-11, -1.990119e-11, 2.933447e-10, -4.291002e-12])

In [141]:
# linear regression non-decision time
y = par_t
X = reg_independent_diff
X = sm.add_constant(X)
# Note the difference in argument order
model_t = sm.OLS(y, X).fit()
predictions = model_t.predict(X) # make the predictions by the model
# Print out the statistics
model_t.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.168
Method:                 Least Squares   F-statistic:                   0.06743
Date:                Sun, 14 Mar 2021   Prob (F-statistic):              0.935
Time:                        10:26:49   Log-Likelihood:                 31.947
No. Observations:                  14   AIC:                            -57.89
Df Residuals:                      11   BIC:                            -55.98
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0194      0.013     -1.548      0.150      -0.047       0.008
erp         8286.7188   2.52e+04      0.329      0.748   -4.71e+04    6.36e+04
alpha       1.225e+07   1.22e+08      0.100      0.922   -2.56e+08    2.81e+08
==============================================================================
Omnibus:                        0.947   Durbin-Watson:                   2.563
Prob(Omnibus):                  0.623   Jarque-Bera (JB):                0.718
Skew:                           0.496   Prob(JB):                        0.698
Kurtosis:                       2.504   Cond. No.                     1.64e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.22e-20. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [142]:
#plot main effect, ccp
fig = sm.graphics.plot_ccpr(model_t, "erp")
fig.tight_layout(pad=1.0)

In [144]:
fig = sm.graphics.plot_ccpr(model_t, "alpha")
fig.tight_layout(pad=1.0)

No handles with labels found to put in legend.


In [23]:
#defference reaction time and accuracy
diff_acc = np.array([ 0.02428127,  2.28873239, -0.91701287,  0.03399378,  0.70921986,
         1.43307397, -0.63936064,  8.36698513,  2.13729932, -0.85513078,
        -2.2713258 , -2.77777778,  3.54082537,  4.38426821])
diff_rt = np.array([ -19.75610227,  -24.30786221,   17.97791128,  -15.82163545,
         -36.99108643,  -25.74826506,  -34.93650585,  -17.63462144,
         -20.26893915,   -8.51105231,  -28.80825474,  -16.5348064 ,
        -103.9004591 ,   -7.49084192])

In [24]:
#response time regression response time 
y = diff_rt
X = reg_independent_diff
X = sm.add_constant(X)
# Note the difference in argument order
model_per = sm.OLS(y, X).fit()
predictions = model_per.predict(X) # make the predictions by the model
# Print out the statistics
model_per.summary()

C:\Users\amin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\amin\Anaconda3\lib\site-packages\scipy\stats\stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.275
Method:                 Least Squares   F-statistic:                     3.469
Date:                Mon, 15 Mar 2021   Prob (F-statistic):             0.0679
Time:                        18:35:30   Log-Likelihood:                -61.830
No. Observations:                  14   AIC:                             129.7
Df Residuals:                      11   BIC:                             131.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.6572     10.140     -1.150      0.275     -33.976      10.662
erp         8.253e+06   2.04e+07      0.405      0.694   -3.66e+07    5.32e+07
alpha      -1.643e+11   9.89e+10     -1.661      0.125   -3.82e+11    5.35e+10
==============================================================================
Omnibus:                       11.862   Durbin-Watson:                   2.216
Prob(Omnibus):                  0.003   Jarque-Bera (JB):                7.870
Skew:                           1.351   Prob(JB):                       0.0195
Kurtosis:                       5.489   Cond. No.                     1.64e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.22e-20. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [25]:
fig = sm.graphics.plot_ccpr(model_per, "erp")
fig.tight_layout(pad=1.0)

In [26]:
fig = sm.graphics.plot_ccpr(model_per, "alpha")
fig.tight_layout(pad=1.0)

In [138]:
#response time regression accuracy
y = diff_acc
X = reg_independent_diff
X = sm.add_constant(X)
# Note the difference in argument order
model_per = sm.OLS(y, X).fit()
predictions = model_per.predict(X) # make the predictions by the model
# Print out the statistics
model_per.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.9025
Date:                Sun, 14 Mar 2021   Prob (F-statistic):              0.434
Time:                        08:40:28   Log-Likelihood:                -33.353
No. Observations:                  14   AIC:                             72.71
Df Residuals:                      11   BIC:                             74.62
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1700      1.326      0.882      0.397      -1.749       4.089
erp          1.82e+06   2.67e+06      0.682      0.509   -4.05e+06    7.69e+06
alpha       1.703e+10   1.29e+10      1.316      0.215   -1.14e+10    4.55e+10
==============================================================================
Omnibus:                        7.717   Durbin-Watson:                   2.056
Prob(Omnibus):                  0.021   Jarque-Bera (JB):                4.263
Skew:                           1.266   Prob(JB):                        0.119
Kurtosis:                       3.945   Cond. No.                     1.64e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.22e-20. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""